In [1]:
import pandas as pd

Gonna try a test of grabbing the ICU beds

**Starting with 2019 data for ease of use, I first read in the numeric data. I created new headers because they didn't bother to include them.**

In [2]:
num = pd.read_csv('HOSP10FY2017/hosp10_2017_NMRC.CSV', header=None, names=['RPT_REC_NUM','WKSHT_CD',
                                                                           'LINE_NUM', 'CLMN_NUM','ITM_VAL_NUM'],
                 dtype={'LINE_NUM':str})

In [3]:
num.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM
0,598679,A000000,00100,00200,118737.0
1,598679,A000000,00100,00300,118737.0
2,598679,A000000,00100,00500,118737.0
3,598679,A000000,00100,00700,118737.0
4,598679,A000000,00200,00200,434831.0


In [4]:
num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19397556 entries, 0 to 19397555
Data columns (total 5 columns):
RPT_REC_NUM    int64
WKSHT_CD       object
LINE_NUM       object
CLMN_NUM       object
ITM_VAL_NUM    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 740.0+ MB


**Then, I filter the numeric data to just the line I want: Worksheet S3, line 8, column 2: The number of ICU beds in each report.**

In [5]:
icu_beds = num[(num['WKSHT_CD'] == 'S300001') & (num['CLMN_NUM'] == '00200') & (num.LINE_NUM == '00800')]

In [6]:
icu_beds.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM
2395,598679,S300001,00800,00200,7.0
5502,599181,S300001,00800,00200,9.0
9303,599223,S300001,00800,00200,20.0
12482,599224,S300001,00800,00200,9.0
16491,599225,S300001,00800,00200,32.0


In [7]:
icu_beds.ITM_VAL_NUM.sum()

65747.0

**Adding the RPT dataset allows me to grab the provider number of each hospital, as well as its report status (since this is a newer datset, many have blanks)**

In [8]:
rpt = pd.read_csv('HOSP10FY2017/hosp10_2017_RPT.CSV', header=None, names=['RPT_REC_NUM',
                                                                         'PRVDR_CTRL_TYPE_CD', 'PRVDR_NUM',
                                                                         'Unknown','RPT_STUS_CD',
                                                                         'FY_BGN_DATE',  'FY_END_DATE',
                                                                         'PROC_DT',
                                                                         'INITL_RPT_SW', 'LAST_RPT_SW',
                                                                         'TRNSMTL_NUM', 'FI_NUM',
                                                                         'ADR_VNDR_CD', 'FI_CREAT_DT',
                                                                         'UTIL_CD',
                                                                         'NPR_DT',  'SPEC_IND',
                                                                         'FI_RCPT_DT'],
                 dtype={'PRVDR_NUM':str})

In [9]:
rpt.head()

,RPT_REC_NUM,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,Unknown,RPT_STUS_CD,FY_BGN_DATE,FY_END_DATE,PROC_DT,INITL_RPT_SW,LAST_RPT_SW,TRNSMTL_NUM,FI_NUM,ADR_VNDR_CD,FI_CREAT_DT,UTIL_CD,NPR_DT,SPEC_IND,FI_RCPT_DT
0,598679,2,360109,NaN,1,10/31/2016,12/31/2016,06/20/2017,N,N,I,15201,4,06/15/2017,F,NaN,NaN,06/05/2017
1,599181,2,510013,NaN,1,10/01/2016,12/31/2016,06/26/2017,N,N,I,11401,4,06/16/2017,F,NaN,NaN,06/02/2017
2,599223,4,070005,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017
3,599224,4,070012,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017
4,599225,4,070027,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017


Unfortunately, there are dupes! Hmmmm...

In [17]:
rpt.pivot_table(index='PRVDR_NUM', values='RPT_REC_NUM', aggfunc=len).sort_values(by='RPT_REC_NUM', ascending=False).head()

,RPT_REC_NUM
PRVDR_NUM,
370025,3
371311,3
313032,2
370237,2
323028,2


It turns out quite a few have ones from 2016, let's eliminate those?

In [20]:
rpt['Year'] = rpt.FY_END_DATE.str[-4:]
rpt.head()

,RPT_REC_NUM,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,Unknown,RPT_STUS_CD,FY_BGN_DATE,FY_END_DATE,PROC_DT,INITL_RPT_SW,LAST_RPT_SW,TRNSMTL_NUM,FI_NUM,ADR_VNDR_CD,FI_CREAT_DT,UTIL_CD,NPR_DT,SPEC_IND,FI_RCPT_DT,Year
0,598679,2,360109,NaN,1,10/31/2016,12/31/2016,06/20/2017,N,N,I,15201,4,06/15/2017,F,NaN,NaN,06/05/2017,2016
1,599181,2,510013,NaN,1,10/01/2016,12/31/2016,06/26/2017,N,N,I,11401,4,06/16/2017,F,NaN,NaN,06/02/2017,2016
2,599223,4,070005,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017,2016
3,599224,4,070012,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017,2016
4,599225,4,070027,NaN,1,10/01/2016,12/31/2016,06/27/2017,N,N,I,13001,4,06/22/2017,F,NaN,NaN,05/31/2017,2016


Pick the later of the two from each provider number

In [24]:
rpt_top = rpt.sort_values('Year',ascending = False).groupby('PRVDR_NUM').head(1)
rpt_top.pivot_table(index='PRVDR_NUM', values='RPT_REC_NUM', aggfunc=len).sort_values(by='RPT_REC_NUM', ascending=False).head()

,RPT_REC_NUM
PRVDR_NUM,
010001,1
360358,1
361302,1
361301,1
361300,1


Merging with the ICU beds dataset:

In [26]:
icu_rpt = icu_beds.merge(rpt_top, on='RPT_REC_NUM', how='left')

icu_rpt.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,Unknown,RPT_STUS_CD,FY_BGN_DATE,...,LAST_RPT_SW,TRNSMTL_NUM,FI_NUM,ADR_VNDR_CD,FI_CREAT_DT,UTIL_CD,NPR_DT,SPEC_IND,FI_RCPT_DT,Year
0,598679,S300001,00800,00200,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,599181,S300001,00800,00200,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,599223,S300001,00800,00200,20.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,599224,S300001,00800,00200,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,599225,S300001,00800,00200,32.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
icu_rpt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3098 entries, 0 to 3097
Data columns (total 23 columns):
RPT_REC_NUM           3098 non-null int64
WKSHT_CD              3098 non-null object
LINE_NUM              3098 non-null object
CLMN_NUM              3098 non-null object
ITM_VAL_NUM           3098 non-null float64
PRVDR_CTRL_TYPE_CD    3048 non-null float64
PRVDR_NUM             3048 non-null object
Unknown               0 non-null float64
RPT_STUS_CD           3048 non-null float64
FY_BGN_DATE           3048 non-null object
FY_END_DATE           3048 non-null object
PROC_DT               3048 non-null object
INITL_RPT_SW          3047 non-null object
LAST_RPT_SW           3047 non-null object
TRNSMTL_NUM           3048 non-null object
FI_NUM                3048 non-null float64
ADR_VNDR_CD           3048 non-null float64
FI_CREAT_DT           3048 non-null object
UTIL_CD               3048 non-null object
NPR_DT                191 non-null object
SPEC_IND              0 non-null

**Next, adding name, characteristics etc. of each hospital to the dataset. This is based on the provider number.**


In [12]:
hc = pd.read_csv('Hospital_General_Information.csv')

In [13]:
hc.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location
0,031309,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private,...,5.0,Not Available,16.0,Not Available,16.0,Not Available,5.0,Not Available,5.0,NaN
1,044021,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-90.175858 35.152962)
2,100277,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private,...,5.0,Not Available,16.0,Not Available,5.0,Not Available,16.0,Not Available,16.0,POINT (-80.191767 25.822972)
3,104078,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-82.612355 27.460155)
4,130063,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary,...,5.0,Above the national average,NaN,Not Available,5.0,Not Available,16.0,Not Available,16.0,POINT (-116.290332 43.611289)


In [28]:
icu_info = icu_rpt.merge(hc, left_on='PRVDR_NUM', right_on='Facility ID')

icu_info.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,Unknown,RPT_STUS_CD,FY_BGN_DATE,...,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location
0,604167,S300001,00800,00200,9.0,4.0,250025,NaN,1.0,01/01/2017,...,NaN,Same as the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN,POINT (-88.477625 33.977551)
1,604212,S300001,00800,00200,8.0,2.0,160001,NaN,1.0,01/01/2017,...,NaN,Below the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,POINT (-92.905914 42.04805)
2,613943,S300001,00800,00200,16.0,4.0,100217,NaN,5.0,10/01/2016,...,NaN,Below the national average,NaN,Same as the national average,NaN,Below the national average,NaN,Below the national average,NaN,POINT (-80.488128 27.844359)
3,618974,S300001,00800,00200,4.0,13.0,660001,NaN,1.0,10/01/2016,...,NaN,Not Available,16.0,Not Available,16.0,Not Available,16.0,Not Available,16.0,NaN
4,619681,S300001,00800,00200,7.0,2.0,360121,NaN,1.0,10/01/2016,...,NaN,Above the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN,POINT (-84.556824 41.47461)


In [29]:
icu_info.to_csv('Data_snippets/2017_ICU_beds.csv')